# Assuming Equal Priors 

In [1]:
import numpy as np

In [2]:
Mean1=[0.1, 0.8]
Mean2=[0.5, 0.2]
cov1=[[1,0],[0,1]]
cov2=[[1, 1.5],[0, -1]]

In [3]:
train_choice=np.random.choice(2, 50000,p=[0.2,0.8])
test_choice=np.random.choice(2, 10000,p=[0.2,0.8])

In [4]:
train_samples=[]
test_samples=[]
for i in train_choice:
    if(i==0):
        train_samples.append([np.random.multivariate_normal(Mean1, cov1),0])
    else:
        train_samples.append([np.random.multivariate_normal(Mean2, cov2),1])

for i in test_choice:
    if(i==0):
        test_samples.append([np.random.multivariate_normal(Mean1, cov1),0])
    else:
        test_samples.append([np.random.multivariate_normal(Mean2, cov2),1])            

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  del sys.path[0]


In [5]:
def variance(x):
    m=np.average(x)
    dev=x-m
    sqr=dev**2
    return np.sum(sqr)/len(x)
def corrcoeff(x, y):
    m1=np.average(x)
    m2=np.average(y)
    dev1=x-m1
    dev2=y-m2
    dev=dev1*dev2
    corr=np.sum(dev)/len(x)
    den=np.sqrt(variance(x))*np.sqrt(variance(y))
    return corr/den
def covariance_matrix(x, y):
    cov=[[None, None],[None, None]]
    cov[0][0]=variance(x)
    cov[1][1]=variance(y)
    cov[1][0]=cov[0][1]=corrcoeff(x, y)
    return cov

In [6]:
x1=[]
y1=[]
x2=[]
y2=[]
for i in train_samples:
    if(i[1]==0):
        x1.append(i[0][0])
        y1.append(i[0][1])
    else:
        x2.append(i[0][0])
        y2.append(i[0][1])
print(np.corrcoef(np.array([x1, y1])))
cov1=covariance_matrix(np.array(x1), np.array(y1))
cov2=covariance_matrix(np.array(x2), np.array(y2))
mean=[None, None]
mean[0]=np.array([np.average(x1), np.average(y1)]).reshape(2,1)
mean[1]=np.array([np.average(x2), np.average(y2)]).reshape(2,1)
print(mean)
        

[[1.         0.00374513]
 [0.00374513 1.        ]]
[array([[0.09676439],
       [0.79525987]]), array([[0.50089865],
       [0.20243306]])]


In [7]:
priors=[len(x1)/50000, len(x2)/50000]
print(priors)

[0.19956, 0.80044]


In [8]:
det=[np.linalg.det(cov1), np.linalg.det(cov2)]
print(det)

[1.0069737365925593, 1.0894828576016826]


In [9]:
invcov1=np.linalg.inv(np.matrix(cov1))
invcov2=np.linalg.inv(np.matrix(cov2))
inverse_cov=[invcov1, invcov2]

In [10]:
a=np.array([1,2]).reshape([1,2])
print(a.shape)
a1=np.dot(a, invcov1)
print(a1)
print(np.dot(a1,np.transpose(a)))

(1, 2)
[[0.99077125 1.98601994]]
[[4.96281113]]


In [11]:
t=np.array(test_samples[0]).reshape(-1,1)
print(t.shape)

(2, 1)


In [12]:
invcov1.shape

(2, 2)

In [13]:
a=np.array([[1,2],[3,4]])
b=np.array([5,6])
print(a.shape)
print(b.shape)
print(np.dot(b,a))

(2, 2)
(2,)
[23 34]


In [14]:
np.log(priors)

array([-1.61164034, -0.2225937 ])

In [15]:
test_samples[0]

[array([-1.62201011, -0.99401699]), 1]

In [16]:
x=np.array(test_samples[67][0]).reshape(-1,1)
print(x.shape)
print(-np.log(det[0])/2-(np.dot(np.dot(x.T, invcov1), x))-2*np.dot(np.dot(x.T, invcov1), np.array(mean[0]))
+np.dot(np.dot(np.array(mean[0]).T, invcov1),np.array(mean[0])))
print(-np.log(det[0])/2-((np.dot(np.dot(x.T, invcov1), x))-2*np.dot(np.dot(x.T, invcov1), np.array(mean[0]))
                               +np.dot(np.dot(np.array(mean[0]).T, invcov1),np.array(mean[0])))/2+np.log(priors[0]))


    

(2, 1)
[[-1.17922757]]
[[-2.01067468]]


# Distinct Covariance Matrix


In [17]:
def descriminant1(dets, sample, mean, prior, invs):
    x=np.reshape(sample, [1,2])
    ans= -np.log(dets)/2-(np.dot(np.dot(x, invs), np.transpose(x))-2*np.dot(np.dot(x, invs),mean)+np.dot(np.dot(np.transpose(mean), invs),mean))/2+np.log(prior)
    return ans                   

In [18]:
pred=[]
maximum=-1

for i in test_samples:
    c=0
    maximum=descriminant1(det[0], i[0], mean[0], priors[0], inverse_cov[0])
    if(descriminant1(det[1], i[0], mean[1], priors[1], inverse_cov[1])>maximum):
        maximum=descriminant1(det[1], i[0], mean[1], priors[1], inverse_cov[1])
        c=1
    pred.append(c)
            
            

In [19]:
acc1=[0,0]
for i in range(len(pred)):
    if(pred[i]==test_samples[i][1]):
        acc1[0]+=1
    else:
        acc1[1]+=1

In [20]:
print(acc1[0]/10000, "% are predicted correctly and", acc1[1]/10000, "% are mispredicted") 

0.8262 % are predicted correctly and 0.1738 % are mispredicted


# Common Variance Matrix 

In [21]:
S=priors[0]*inverse_cov[0]+priors[1]*inverse_cov[1]

In [22]:
def descriminant2(dets, sample, mean, prior, invs):
    x=np.reshape(sample, [1,2])
    ans= -(np.dot(np.dot(x, invs), np.transpose(x))-2*np.dot(np.dot(x, invs),mean)+np.dot(np.dot(np.transpose(mean), invs),mean))/2+np.log(prior)
    return ans                   

In [23]:
pred=[]
maximum=-1

for i in test_samples:
    c=0
    maximum=descriminant1(det[0], i[0], mean[0], priors[0], S)
    if(descriminant1(det[1], i[0], mean[1], priors[1], S)>maximum):
        maximum=descriminant1(det[1], i[0], mean[1], priors[1], S)
        c=1
    pred.append(c)
            
            

In [24]:
acc1=[0,0]
for i in range(len(pred)):
    if(pred[i]==test_samples[i][1]):
        acc1[0]+=1
    else:
        acc1[1]+=1

In [25]:
print(acc1[0]/10000, "% are predicted correctly and", acc1[1]/10000, "% are mispredicted") 

0.8219 % are predicted correctly and 0.1781 % are mispredicted


# Common Covariance with offdiagonals zero 

In [26]:
SD=np.array(S)
SD[0][1]=SD[1][0]=0

In [27]:
def descriminant3(dets, sample, mean, prior, invs):
    x=np.reshape(sample, [2])
    ans= -((((sample[0]-mean[0])/invs[0][0])**2)+(((sample[0]-mean[0])/invs[0][0])**2))/2+np.log(prior)
    return ans                   

In [28]:
pred=[]
maximum=-1

for i in test_samples:
    c=0
    maximum=descriminant3(det[0], i[0], mean[0], priors[0], SD)
    if(descriminant3(det[1], i[0], mean[1], priors[1], SD)>maximum):
        maximum=descriminant3(det[1], i[0], mean[1], priors[1], SD)
        c=1
    pred.append(c)
            
            

In [29]:
acc1=[0,0]
for i in range(len(pred)):
    if(pred[i]==test_samples[i][1]):
        acc1[0]+=1
    else:
        acc1[1]+=1

In [30]:
print(acc1[0]/10000, "% are predicted correctly and", acc1[1]/10000, "% are mispredicted") 

0.799 % are predicted correctly and 0.201 % are mispredicted


# Common Covariance and equal priors


In [31]:
def descriminant4(sample, mean):
    x=np.reshape(sample, [2,1])
    mean=np.reshape(np.array(mean), [2,1])
    ans= -(np.dot((x-mean).T,(x-mean)))
    return ans                   

In [32]:
pred=[]
maximum=-1

for i in test_samples:
    c=0
    maximum=descriminant4(i[0], mean[0])
    if(descriminant4( i[0], mean[1])>maximum):
        c=1
    pred.append(c)
            
            

In [33]:
pred=[]
maximum=-1

for i in test_samples:
    c=0
    maximum=descriminant3(det[0], i[0], mean[0], priors[0], SD)
    if(descriminant3(det[1], i[0], mean[1], priors[1], SD)>maximum):
        maximum=descriminant3(det[1], i[0], mean[1], priors[1], SD)
        c=1
    pred.append(c)
            
            

In [34]:
acc1=[0,0]
for i in range(len(pred)):
    if(pred[i]==test_samples[i][1]):
        acc1[0]+=1
    else:
        acc1[1]+=1

In [35]:
print(acc1[0]/10000, "% are predicted correctly and", acc1[1]/10000, "% are mispredicted") 

0.799 % are predicted correctly and 0.201 % are mispredicted
